In [111]:
import pandas as pd
import geopandas as gpd
import numpy as np
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import *
import psycopg2
import time
%matplotlib inline

pd.options.display.max_rows = 999
pd.options.display.max_columns = None

In [112]:
password = open('password.txt','r').readline()
username = 'kmerrem'

#this is to write from geopandas to postgis table 
#gisdata_engine = create_engine('postgresql://%s:%s@172.19.118.205:5432/gisdata' %(username,password))
walkability_engine = create_engine('postgresql://%s:%s@172.19.118.205:5432/walkability' %(username,password))

In [113]:
#buildings = gpd.read_postgis("select * from staging.buildings322", walkability_engine, 'geom')
#buildings = gpd.read_postgis("select * from staging.buildings327", walkability_engine, 'geom')
cts = gpd.read_postgis("SELECT * FROM public.census_tracts_blank", walkability_engine, 'geom')
#pednet = gpd.read_postgis("select * from compliance.att_pednet_all_in WHERE geom IS NOT NULL", walkability_engine, 'geom')
#pednet = gpd.read_postgis("select * from staging.a_z1_pednet WHERE geom IS NOT NULL", walkability_engine, 'geom')
pednet = gpd.read_postgis("select * from final.pednet WHERE geom IS NOT NULL", walkability_engine, 'geom')

In [114]:
pednet.head(2)

,gid_old,__gid,edge_id,start_node,end_node,next_left_,abs_next_l,next_right,abs_next_r,left_face,right_face,gid_centr,centreline,linear_nam,linear_n_1,linear_n_2,linear_n_3,linear_n_4,address_l,address_r,parity_l,parity_r,lo_num_l,lo_num_r,hi_num_l,hi_num_r,from_inter,to_interse,oneway_dir,oneway_d_1,feature_co,feature__1,pt2,shortest_d,load_date,load_user,data_autho,obs_pts,road_type,sdwlk_desc,sdwlk_code,gcc_sdwlk_,gcc_sdwl_1,topo_sdwlk,qa_sdwlk_c,crosswalk,geom,last_change_timestamp,last_change_user,last_change_ipaddr,last_used_application,qa_sdwlk_code,qa_sdwlk_code1,crosswalk_boolean,qa_sdwlk_code2,gid,zid,f_cwalk_v1,f_cwalk,px,px_type,cwalk_type,length
0,35993,63263.0,68198.0,42281.0,4110.0,64844.0,64844.0,-49921.0,49921.0,19376.0,19375.0,1025.0,913257.0,1935.0,Kipling Ave,Kipling Avenue,Kipling,Ave,710-740,717-727,E,O,710.0,717.0,740.0,727.0,13469594.0,13469526.0,0.0,Not One-Way,201200.0,Major Arterial,SRID=2019;POINT(302472.57 4831540.274),11.0,2019-01-15,gepure,gepure,None,Major Arterial,Sidewalk on east side; partially on other side,5.0,NaN,None,NaN,None,None,(LINESTRING (302473.0485320848 4831586.4744568...,2019-03-14 20:53:53.000000,djokic,172.19.116.90/32,QGIS,None,Pass,True,None,86713,35414,None,None,None,None,None,120.1097
1,3034,28596.0,65343.0,52760.0,52761.0,65344.0,65344.0,-65342.0,65342.0,13986.0,13325.0,43820.0,11415001.0,6235.0,Moatfield Dr,Moatfield Drive,Moatfield,Dr,2-100,85-125,E,O,2.0,85.0,100.0,125.0,13448747.0,13448975.0,0.0,Not One-Way,201400.0,Collector,SRID=2019;POINT(316735.327898563 4846509.78786...,4.0,2019-01-15,gepure,gepure,None,Collector,Sidewalk on both sides,7.0,NaN,None,NaN,None,Y,"(LINESTRING (316713.1928 4846504.7597, 316776....",2019-02-21 10:52:40.531515,postgres,172.19.116.225/32,PostgreSQL JDBC Driver,None,Pass,True,None,83773,52988,,,None,None,,68.0376


In [115]:
pednet.dtypes

gid_old                           int64
__gid                           float64
edge_id                         float64
start_node                      float64
end_node                        float64
next_left_                      float64
abs_next_l                      float64
next_right                      float64
abs_next_r                      float64
left_face                       float64
right_face                      float64
gid_centr                       float64
centreline                      float64
linear_nam                      float64
linear_n_1                       object
linear_n_2                       object
linear_n_3                       object
linear_n_4                       object
address_l                        object
address_r                        object
parity_l                         object
parity_r                         object
lo_num_l                        float64
lo_num_r                        float64
hi_num_l                        float64


In [ ]:
#jspatial joining of pednet to get ctnum on pednet segments
pednet_ct = gpd.sjoin(pednet,cts)


In [ ]:
pednet_ct.head(2)

In [82]:
#NEED THE INDEX TO RUN
cts =cts.set_index('ctnum')
#summary_buildings = summary_buildings.set_index('ctnum')

In [83]:
cts

,gid,__gid,ctuid,ctname,pruid,prname,cmauid,cmapuid,cmaname,cmatype,shape_leng,shape_area,geotext,geom
ctnum,,,,,,,,,,,,,,
535000100.0,1,62,5350001.00,0001.00,35,Ontario,535,35535,Toronto,B,36906.495474,6.619690e+06,535000100,(POLYGON ((320588.9306980086 4835855.647789237...
535000200.0,2,63,5350002.00,0002.00,35,Ontario,535,35535,Toronto,B,32534.307654,3.293422e+06,535000200,"(POLYGON ((316642.831678931 4832470.554474635,..."
535026400.0,3,87,5350264.00,0264.00,35,Ontario,535,35535,Toronto,B,12241.116870,3.464007e+06,535026400,(POLYGON ((314351.8668838083 4845056.100542194...
535008200.0,4,94,5350082.00,0082.00,35,Ontario,535,35535,Toronto,B,2310.602463,3.082930e+05,535008200,(POLYGON ((318379.9191213865 4837764.438619476...
535000600.0,5,105,5350006.00,0006.00,35,Ontario,535,35535,Toronto,B,455.941409,1.287364e+04,535000600,"(POLYGON ((310234.199135254 4832619.302139398,..."
535016300.0,6,217,5350163.00,0163.00,35,Ontario,535,35535,Toronto,B,2842.202163,4.540284e+05,535016300,(POLYGON ((309832.5952238032 4837976.043997317...
535021302.0,7,443,5350213.02,0213.02,35,Ontario,535,35535,Toronto,B,14621.600658,7.041416e+06,535021302,(POLYGON ((302360.8525544122 4831898.581168552...
535037604.0,8,455,5350376.04,0376.04,35,Ontario,535,35535,Toronto,B,4439.479932,1.193620e+06,535037604,(POLYGON ((318932.6039103945 4852366.218676397...
535024701.0,9,501,5350247.01,0247.01,35,Ontario,535,35535,Toronto,B,23043.297794,1.734167e+07,535024701,"(POLYGON ((294567.331474375 4843841.86104348, ..."


In [84]:
pednet_ct['sdwlk_length'] =np.nan
pednet_ct["sdwlk_length"]  =pednet_ct.length*1

# setting length times two when it is a local road and sidewalks code = 7 - both sides
idx = pednet_ct[(pednet_ct.sdwlk_code == 7.0) &(pednet_ct.feature__1 == "Local") ].index
pednet_ct.loc[idx,'sdwlk_length'] =  pednet_ct.loc[idx].length*2

# setting length to zero when it is a local road and sidewalks code = 3 - no sidewalk on either side
idx = pednet_ct[(pednet_ct.sdwlk_code == 3.0) &(pednet_ct.feature__1 == "Local") ].index
pednet_ct.loc[idx,'sdwlk_length'] =  pednet_ct.loc[idx].length*0

# setting length to zero when it is a local road and sidewalks code = 13 - not applicable
idx = pednet_ct[(pednet_ct.sdwlk_code == 13.0) &(pednet_ct.feature__1 == "Local") ].index
pednet_ct.loc[idx,'sdwlk_length'] =  pednet_ct.loc[idx].length*0

idx = pednet_ct[(pednet_ct.sdwlk_code == 13.0) &(pednet_ct.feature__1 == "Local") ].index
pednet_ct.loc[idx,'sdwlk_length'] =  pednet_ct.loc[idx].length*0


In [85]:
pednet_ct.head(2)

,gid_old,__gid_left,edge_id,start_node,end_node,next_left_,abs_next_l,next_right,abs_next_r,left_face,right_face,gid_centr,centreline,linear_nam,linear_n_1,linear_n_2,linear_n_3,linear_n_4,address_l,address_r,parity_l,parity_r,lo_num_l,lo_num_r,hi_num_l,hi_num_r,from_inter,to_interse,oneway_dir,oneway_d_1,feature_co,feature__1,pt2,shortest_d,load_date,load_user,data_autho,obs_pts,road_type,sdwlk_desc,sdwlk_code,gcc_sdwlk_,gcc_sdwl_1,topo_sdwlk,qa_sdwlk_c,crosswalk,geom,last_change_timestamp,last_change_user,last_change_ipaddr,last_used_application,qa_sdwlk_code,qa_sdwlk_code1,crosswalk_boolean,qa_sdwlk_code2,gid_left,zid,f_cwalk_v1,f_cwalk,px,px_type,cwalk_type,index_right,gid_right,__gid_right,ctuid,ctname,pruid,prname,cmauid,cmapuid,cmaname,cmatype,shape_leng,shape_area,ctnum,geotext,sdwlk_length
0,703,37494.0,10000.0,8896.0,8894.0,30550.0,30550.0,-6384.0,6384.0,15795.0,676.0,68336.0,1146460.0,3803.0,Lake Shore Blvd E,Lake Shore Boulevard East,Lake Shore,Blvd,None,25-25,N,O,0.0,25.0,0.0,25.0,13467661.0,13467576.0,0.0,Against-Digitization,201200.0,Major Arterial,SRID=2019;POINT(314818.420913779 4833584.71111...,0.0,2019-01-15,gepure,gepure,None,Major Arterial,No sidewalk on either side,3.0,0.0,None,0.0,None,None,"(LINESTRING (314815.8607 4833592.2401, 314821....",2019-02-20 17:43:11.671090,postgres,127.0.0.1/32,,None,Pass,True,None,34488,30665,None,Y,211,ts,ts,280,281,265,5350013.02,0013.02,35,Ontario,535,35535,Toronto,B,3525.22685,417800.246307,535001302.0,535001302,17.089354
13,40429,37459.0,6385.0,8895.0,8893.0,6383.0,6383.0,-64927.0,64927.0,676.0,15789.0,34443.0,1146489.0,1962.0,Lake Shore Blvd W,Lake Shore Boulevard West,Lake Shore,Blvd,None,None,N,N,0.0,0.0,0.0,0.0,13467661.0,13467693.0,1.0,Follow-Digitization,201200.0,Major Arterial,SRID=2019;POINT(314794.688721227 4833568.09292...,0.0,2019-01-15,gepure,gepure,None,Major Arterial,Sidewalk on both sides,7.0,NaN,None,NaN,None,None,"(LINESTRING (314792.3111 4833576.0342, 314797....",2019-02-20 17:43:11.671090,postgres,127.0.0.1/32,,None,Pass,True,None,22847,18932,None,Y,211,ts,ts,280,281,265,5350013.02,0013.02,35,Ontario,535,35535,Toronto,B,3525.22685,417800.246307,535001302.0,535001302,17.465946


In [86]:
#ratio_crosswalk = crosswalk segements = True / Total segments (True and False) e.g. 15 True / (15 True + 300 False) x 100 
for group, df in pednet_ct.groupby('ctnum'):
    cts.loc[group,"num_crosswalks"]  =df[df.crosswalk_boolean == True].crosswalk_boolean.count()
    cts.loc[group,"tot_crosswalk_leng"]  =df[df.crosswalk_boolean == True].length.sum()
    cts.loc[group,"ratio_crosswalk_leng"]  =df[df.crosswalk_boolean == True].length.sum()/df[df.crosswalk_boolean == False].length.sum()*100
    cts.loc[group,"ratio_crosswalks"]  =df[df.crosswalk_boolean == True].crosswalk_boolean.count()/df.crosswalk_boolean.count()*100
    cts.loc[group,"avg_crosswalk_leng"]  =df[df.crosswalk_boolean == True].length.mean()
    cts.loc[group,"med_crosswalk_leng"]  =df[df.crosswalk_boolean == True].length.median()
    cts.loc[group,"max_crosswalk_leng"]  =df[df.crosswalk_boolean == True].length.max()
    cts.loc[group,"sdwlk_length"]  =df.sdwlk_length.sum()

In [87]:
cts.head(2)

,gid,__gid,ctuid,ctname,pruid,prname,cmauid,cmapuid,cmaname,cmatype,shape_leng,shape_area,geotext,geom,num_crosswalks,tot_crosswalk_leng,ratio_crosswalk_leng,ratio_crosswalks,avg_crosswalk_leng,med_crosswalk_leng,max_crosswalk_leng,sdwlk_length
ctnum,,,,,,,,,,,,,,,,,,,,,,
535000100.0,1,62,5350001.00,0001.00,35,Ontario,535,35535,Toronto,B,36906.495474,6.619690e+06,535000100,(POLYGON ((320588.9306980086 4835855.647789237...,48.0,1246.698385,2.889998,12.339332,25.972883,24.39922,50.59,41566.585621
535000200.0,2,63,5350002.00,0002.00,35,Ontario,535,35535,Toronto,B,32534.307654,3.293422e+06,535000200,"(POLYGON ((316642.831678931 4832470.554474635,...",0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,25280.460065


In [88]:
df[df.crosswalk_boolean == True].crosswalk_boolean.count()/df.crosswalk_boolean.count()*100

3.5856573705179287

In [89]:
df[df.crosswalk_boolean == True].length.sum()/df.length.sum()*100

0.8084307290242035

In [90]:
#crosswalk count
pednet_ct[pednet_ct.crosswalk_boolean == True].crosswalk_boolean.count()

12230

In [95]:
cts.head(2)

,gid,__gid,ctuid,ctname,pruid,prname,cmauid,cmapuid,cmaname,cmatype,shape_leng,shape_area,geotext,geom,num_crosswalks,tot_crosswalk_leng,ratio_crosswalk_leng,ratio_crosswalks,avg_crosswalk_leng,med_crosswalk_leng,max_crosswalk_leng,sdwlk_length
ctnum,,,,,,,,,,,,,,,,,,,,,,
535000100.0,1,62,5350001.00,0001.00,35,Ontario,535,35535,Toronto,B,36906.495474,6.619690e+06,535000100,(POLYGON ((320588.9306980086 4835855.647789237...,48.0,1246.698385,2.889998,12.339332,25.972883,24.39922,50.59,41566.585621
535000200.0,2,63,5350002.00,0002.00,35,Ontario,535,35535,Toronto,B,32534.307654,3.293422e+06,535000200,"(POLYGON ((316642.831678931 4832470.554474635,...",0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,25280.460065


In [91]:
cts1

,gid,__gid,ctuid,ctname,pruid,prname,cmauid,cmapuid,cmaname,cmatype,shape_leng,shape_area,geotext,geom,num_crosswalks,tot_crosswalk_leng,ratio_crosswalk_leng,ratio_crosswalks,avg_crosswalk_leng,med_crosswalk_leng,max_crosswalk_leng,sdwlk_length,objectid
ctnum,,,,,,,,,,,,,,,,,,,,,,,
535000100.0,1,62,5350001.00,0001.00,35,Ontario,535,35535,Toronto,B,36906.495474,6.619690e+06,535000100,MULTIPOLYGON (((320588.9306980086 4835855.6477...,48.0,1246.698385,2.889998,12.339332,25.972883,24.399220,50.590000,41566.585621,535000100
535000200.0,2,63,5350002.00,0002.00,35,Ontario,535,35535,Toronto,B,32534.307654,3.293422e+06,535000200,MULTIPOLYGON (((316642.831678931 4832470.55447...,0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,25280.460065,535000200
535026400.0,3,87,5350264.00,0264.00,35,Ontario,535,35535,Toronto,B,12241.116870,3.464007e+06,535026400,MULTIPOLYGON (((314351.8668838083 4845056.1005...,9.0,233.733311,0.958363,4.072398,25.970368,26.419112,31.194496,25103.321279,535026400
535008200.0,4,94,5350082.00,0082.00,35,Ontario,535,35535,Toronto,B,2310.602463,3.082930e+05,535008200,MULTIPOLYGON (((318379.9191213865 4837764.4386...,8.0,161.986789,2.520676,12.698413,20.248349,20.976387,23.591011,10388.049895,535008200
535000600.0,5,105,5350006.00,0006.00,35,Ontario,535,35535,Toronto,B,455.941409,1.287364e+04,535000600,MULTIPOLYGON (((310234.199135254 4832619.30213...,0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,1275.167664,535000600
535016300.0,6,217,5350163.00,0163.00,35,Ontario,535,35535,Toronto,B,2842.202163,4.540284e+05,535016300,MULTIPOLYGON (((309832.5952238032 4837976.0439...,7.0,113.803716,1.425627,10.769231,16.257674,16.863825,18.898599,13758.404982,535016300
535021302.0,7,443,5350213.02,0213.02,35,Ontario,535,35535,Toronto,B,14621.600658,7.041416e+06,535021302,MULTIPOLYGON (((302360.8525544122 4831898.5811...,116.0,3361.451653,7.078200,24.733475,28.978031,29.606072,51.861773,45568.426450,535021302
535037604.0,8,455,5350376.04,0376.04,35,Ontario,535,35535,Toronto,B,4439.479932,1.193620e+06,535037604,MULTIPOLYGON (((318932.6039103945 4852366.2186...,29.0,744.117349,4.043941,14.795918,25.659219,24.894897,33.370296,23001.757125,535037604
535024701.0,9,501,5350247.01,0247.01,35,Ontario,535,35535,Toronto,B,23043.297794,1.734167e+07,535024701,MULTIPOLYGON (((294567.331474375 4843841.86104...,114.0,3221.176865,3.186832,15.468114,28.255937,28.275611,45.639928,99304.349598,535024701


In [92]:
#cts1 = cts1[cts1.geom.notna()]
#cts1

In [93]:
#You always need to drop the geometry column and write a new WKT text geometry column 

cts1 = cts.copy()
cts1 = cts1[cts1.geom.notna()]
cts1['geometry'] = cts1['geom']
cts1['geom'] = cts1['geometry'].apply(lambda x: WKTElement(x.wkt, srid=2019))
#drop the geometry column as it is now duplicative
cts1.drop('geometry', 1, inplace=True)
#Adds objectid as double. Need to change to integer
cts1['objectid'] = cts1.index.astype(np.int64)
cts1.to_sql('census_summary_cross_v2', walkability_engine, schema='results', if_exists='replace', index=True, 
                         dtype={'geom': Geometry('MULTIPOLYGON', srid=2019)})

In [94]:
cts1

,gid,__gid,ctuid,ctname,pruid,prname,cmauid,cmapuid,cmaname,cmatype,shape_leng,shape_area,geotext,geom,num_crosswalks,tot_crosswalk_leng,ratio_crosswalk_leng,ratio_crosswalks,avg_crosswalk_leng,med_crosswalk_leng,max_crosswalk_leng,sdwlk_length,objectid
ctnum,,,,,,,,,,,,,,,,,,,,,,,
535000100.0,1,62,5350001.00,0001.00,35,Ontario,535,35535,Toronto,B,36906.495474,6.619690e+06,535000100,MULTIPOLYGON (((320588.9306980086 4835855.6477...,48.0,1246.698385,2.889998,12.339332,25.972883,24.399220,50.590000,41566.585621,535000100
535000200.0,2,63,5350002.00,0002.00,35,Ontario,535,35535,Toronto,B,32534.307654,3.293422e+06,535000200,MULTIPOLYGON (((316642.831678931 4832470.55447...,0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,25280.460065,535000200
535026400.0,3,87,5350264.00,0264.00,35,Ontario,535,35535,Toronto,B,12241.116870,3.464007e+06,535026400,MULTIPOLYGON (((314351.8668838083 4845056.1005...,9.0,233.733311,0.958363,4.072398,25.970368,26.419112,31.194496,25103.321279,535026400
535008200.0,4,94,5350082.00,0082.00,35,Ontario,535,35535,Toronto,B,2310.602463,3.082930e+05,535008200,MULTIPOLYGON (((318379.9191213865 4837764.4386...,8.0,161.986789,2.520676,12.698413,20.248349,20.976387,23.591011,10388.049895,535008200
535000600.0,5,105,5350006.00,0006.00,35,Ontario,535,35535,Toronto,B,455.941409,1.287364e+04,535000600,MULTIPOLYGON (((310234.199135254 4832619.30213...,0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,1275.167664,535000600
535016300.0,6,217,5350163.00,0163.00,35,Ontario,535,35535,Toronto,B,2842.202163,4.540284e+05,535016300,MULTIPOLYGON (((309832.5952238032 4837976.0439...,7.0,113.803716,1.425627,10.769231,16.257674,16.863825,18.898599,13758.404982,535016300
535021302.0,7,443,5350213.02,0213.02,35,Ontario,535,35535,Toronto,B,14621.600658,7.041416e+06,535021302,MULTIPOLYGON (((302360.8525544122 4831898.5811...,116.0,3361.451653,7.078200,24.733475,28.978031,29.606072,51.861773,45568.426450,535021302
535037604.0,8,455,5350376.04,0376.04,35,Ontario,535,35535,Toronto,B,4439.479932,1.193620e+06,535037604,MULTIPOLYGON (((318932.6039103945 4852366.2186...,29.0,744.117349,4.043941,14.795918,25.659219,24.894897,33.370296,23001.757125,535037604
535024701.0,9,501,5350247.01,0247.01,35,Ontario,535,35535,Toronto,B,23043.297794,1.734167e+07,535024701,MULTIPOLYGON (((294567.331474375 4843841.86104...,114.0,3221.176865,3.186832,15.468114,28.255937,28.275611,45.639928,99304.349598,535024701
